In [14]:
import pandas as pd
import os
import math

In [40]:
###

print("Path at terminal when executing this file")
print(os.getcwd() + "\n")

Path at terminal when executing this file
/Users/gregoryboyer/Documents/DataScience/Code for SF Analytics



## Establish data sources

In [4]:
users = pd.read_csv('../data/user_list_data.csv')
channels = pd.read_csv('../data/conversation_list_data.csv') 
conversations = pd.read_csv('../data/conversations_history_data.csv') 

new_member_form = pd.read_csv('../data/new_member_form.csv')
new_member_form.set_index('Email Address')

print('files loaded')

files loaded


In [5]:


# Separate messages into public_messages and joins
# TODO: there is a messsage type that may be more accurate to use for joins
# TODO: does not include replies, only original messages

public_messages = conversations[(~conversations['text'].str.contains('joined the channel', na=False))]
channel_joins = conversations[(conversations['text'].str.contains('joined the channel', na=False))]


In [6]:

# Find the earliest join message, based on the assumption that when joining slack, 
# this message is automatically sent.
join_date = channel_joins.groupby(['user'], as_index = False).agg({'ts':'min'})
join_date = join_date.rename(columns = {'ts' : 'ts_join_date'})
join_date['join_date'] = join_date['ts_join_date']


# Find earliest message
first_public_message = public_messages.groupby(['user'], as_index = False).agg({'ts':'min'})
first_public_message = first_public_message.rename(columns = {'ts' : 'ts_first_message'})


In [8]:
# Find most recent message
latest_public_message = public_messages.groupby(['user'], as_index = False).agg({'ts':'max'})
latest_public_message = latest_public_message.rename(columns = {'ts' : 'ts_latest_message'})

latest_public_message['latest_message_datedate'] = pd.to_datetime(latest_public_message['ts_latest_message'],unit='s')

In [9]:
# message_count
message_count = public_messages.groupby(['user'], as_index = False).agg({'datetime': pd.Series.count})
message_count = message_count.rename(columns = {'datetime' : 'total_message_count'})





In [10]:
# Total Message days
message_days = public_messages[['user','datetime']]

# TODO: This works with warning
message_days['datetime'] = pd.to_datetime(message_days['datetime']).dt.date

message_days = message_days.groupby(['user'], as_index = False).agg({'datetime': pd.Series.nunique})
message_days = message_days.rename(columns = {'datetime' : 'unique_message_days'})


/Users/gboyer/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
# Create User Summary
user_summary = users

print(users.shape)


user_summary = pd.merge(users, join_date, how = 'left', left_on = ['id'], right_on = ['user'])
user_summary = pd.merge(user_summary, first_public_message[['user','ts_first_message']], how = 'left', left_on = ['id'], right_on = ['user'])
user_summary = pd.merge(user_summary, latest_public_message[['user','ts_latest_message']], how = 'left', left_on = ['id'], right_on = ['user'])

user_summary = pd.merge(user_summary, message_count, how = 'left', left_on = ['id'], right_on = ['user'])
user_summary = pd.merge(user_summary, message_days, how = 'left', left_on = ['id'], right_on = ['user'])


# user_summary

(3711, 11)


In [12]:
user_summary

,Unnamed: 0,id,email,title,first_name,last_name,real_name,tz,display_name,is_email_confirmed,...,ts_join_date,join_date,user_y,ts_first_message,user_x,ts_latest_message,user_y,total_message_count,user,unique_message_days
0,0,USLACKBOT,NaN,NaN,slackbot,NaN,Slackbot,NaN,Slackbot,NaN,...,NaN,NaN,USLACKBOT,1.462993e+09,USLACKBOT,1.642633e+09,USLACKBOT,418.0,USLACKBOT,255.0
1,1,U042ZA3GX,jason.lally@gmail.com,Chief Data Officer at City and County of San F...,Jason,Lally,Jason Lally,NaN,"Jason Lally (State of CA, he/him)",NaN,...,1.415849e+09,1.415849e+09,U042ZA3GX,1.415908e+09,U042ZA3GX,1.633546e+09,U042ZA3GX,643.0,U042ZA3GX,225.0
2,2,U04301E01,julia@codeforsanfrancisco.org,NaN,Julia,Gitis,Julia Gitis,NaN,julia,NaN,...,1.426749e+09,1.426749e+09,U04301E01,1.426834e+09,U04301E01,1.437022e+09,U04301E01,6.0,U04301E01,4.0
3,3,U0431NL8E,jesse@codeforsanfrancisco.org,Former Captain,Jesse,Biroscak,Jesse Biroscak,NaN,jesse,NaN,...,1.426741e+09,1.426741e+09,U0431NL8E,1.427078e+09,U0431NL8E,1.517281e+09,U0431NL8E,278.0,U0431NL8E,114.0
4,4,U043262MY,judyvansoldt@gmail.com,NaN,Judy,van Soldt,Judy van Soldt,NaN,judy,NaN,...,1.426746e+09,1.426746e+09,U043262MY,1.426750e+09,U043262MY,1.458621e+09,U043262MY,62.0,U043262MY,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,3706,U02V64XHC5Q,shreya.gif@gmail.com,NaN,Shreya,Jaiswal,Shreya Jaiswal,NaN,Shreya Jaiswal,NaN,...,1.642868e+09,1.642868e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3707,3707,U02VA9ABAAY,tenda5212002@hotmail.com,NaN,Bebe,NaN,Bebe,NaN,Bebe,NaN,...,1.642983e+09,1.642983e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3708,3708,U02VDRAEFT8,ali.el@outlook.com,NaN,Ali,El,Ali El,NaN,NaN,NaN,...,1.643151e+09,1.643151e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3709,3709,U0300T3UCMA,simon.morfit@gmail.com,NaN,Simon,Morfit,Simon Morfit,NaN,Simon Morfit,NaN,...,1.643049e+09,1.643049e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Add calcs on User Summary

# Find time to first post
user_summary['days_to_first_post'] = (user_summary['ts_first_message'] - user_summary['ts_join_date']) / 60 / 60 / 24

user_summary.days_to_first_post = user_summary.days_to_first_post.apply(lambda x: math.floor(x) if pd.notnull(x) else x )


user_summary.to_csv('./testdata.csv')
user_summary



,Unnamed: 0,id,email,title,first_name,last_name,real_name,tz,display_name,is_email_confirmed,...,join_date,user_y,ts_first_message,user_x,ts_latest_message,user_y,total_message_count,user,unique_message_days,days_to_first_post
0,0,USLACKBOT,NaN,NaN,slackbot,NaN,Slackbot,NaN,Slackbot,NaN,...,NaN,USLACKBOT,1.462993e+09,USLACKBOT,1.642633e+09,USLACKBOT,418.0,USLACKBOT,255.0,NaN
1,1,U042ZA3GX,jason.lally@gmail.com,Chief Data Officer at City and County of San F...,Jason,Lally,Jason Lally,NaN,"Jason Lally (State of CA, he/him)",NaN,...,1.415849e+09,U042ZA3GX,1.415908e+09,U042ZA3GX,1.633546e+09,U042ZA3GX,643.0,U042ZA3GX,225.0,0.0
2,2,U04301E01,julia@codeforsanfrancisco.org,NaN,Julia,Gitis,Julia Gitis,NaN,julia,NaN,...,1.426749e+09,U04301E01,1.426834e+09,U04301E01,1.437022e+09,U04301E01,6.0,U04301E01,4.0,0.0
3,3,U0431NL8E,jesse@codeforsanfrancisco.org,Former Captain,Jesse,Biroscak,Jesse Biroscak,NaN,jesse,NaN,...,1.426741e+09,U0431NL8E,1.427078e+09,U0431NL8E,1.517281e+09,U0431NL8E,278.0,U0431NL8E,114.0,3.0
4,4,U043262MY,judyvansoldt@gmail.com,NaN,Judy,van Soldt,Judy van Soldt,NaN,judy,NaN,...,1.426746e+09,U043262MY,1.426750e+09,U043262MY,1.458621e+09,U043262MY,62.0,U043262MY,39.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,3706,U02V64XHC5Q,shreya.gif@gmail.com,NaN,Shreya,Jaiswal,Shreya Jaiswal,NaN,Shreya Jaiswal,NaN,...,1.642868e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3707,3707,U02VA9ABAAY,tenda5212002@hotmail.com,NaN,Bebe,NaN,Bebe,NaN,Bebe,NaN,...,1.642983e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3708,3708,U02VDRAEFT8,ali.el@outlook.com,NaN,Ali,El,Ali El,NaN,NaN,NaN,...,1.643151e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3709,3709,U0300T3UCMA,simon.morfit@gmail.com,NaN,Simon,Morfit,Simon Morfit,NaN,Simon Morfit,NaN,...,1.643049e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Join demographic info

In [16]:


output = pd.merge(
    user_summary,
    new_member_form,
    how="inner",
    left_on='email',
    right_on='Email Address',

)

In [61]:
output.to_csv('data/user_summary')